# Doc2Vec

上一节讲了Word2Vec可以用来训练词向量，那么句子向量怎么训练呢？

那这一节就看下如何训练句向量。

构建句子向量有比较多的方法，比较流行的是：
- bag-of-words词袋模型
- 基于word2vec计算句子中各个词语的词向量，然后加和平均得到句子向量
- 使用doc2vec训练句向量模型

说下这几个模型的优缺点：
1. 词袋模型
词袋模型存在两个主要的缺点：一个是词袋模型忽略词序，如果两个不同的句子由相同的词但是顺序不同组成，词袋模型会将这两句话定义为同一个表达；另一个是词袋模型忽略了句法，这样训练出来的模型会造成类似‘powerful‘，‘strong’和‘Paris’的距离是相同的，而其实‘powerful’应该相对于’Paris‘距离’strong‘更近才对。

2. word2vec
比较简单，没有考虑句子词语之间的相互语义关系，我写过一个开源实现：[text2vec](https://github.com/shibing624/text2vec)

3. doc2vec
本节的重点，下面介绍doc2vec模型。



Doc2vec又叫Paragraph Vector是Tomas Mikolov基于word2vec模型提出的，其具有一些优点，比如不用固定句子长度，接受不同长度的句子做训练样本，Doc2vec是一个无监督学习算法，该算法用于预测一个向量来表示不同的文档，该模型的结构潜在的克服了词袋模型的缺点。

Doc2vec也有两种训练方式，一种是PV-DM（Distributed Memory Model of paragraph vectors）类似于word2vec中的CBOW模型，另一种是PV-DBOW（Distributed Bag of Words of paragraph vector类似于Word2vec中的skip-gram模型。

In [8]:
import numpy as np

from gensim.models.doc2vec import Doc2Vec, TaggedDocument


def get_datasest():
    with open("data/C000008_test.txt", 'r', encoding='utf8') as cf:
        docs = cf.readlines()
        print('docs size:', len(docs))

    x_train = []

    for i, text in enumerate(docs):
        word_list = text.split()
        l = len(word_list)
        word_list[l - 1] = word_list[l - 1].strip()
        document = TaggedDocument(word_list, tags=[i])
        x_train.append(document)

    return x_train


def get_vecs(model, corpus, size):
    vecs = [np.array(model.docvecs[z.tags[0]].reshape(1, size))
            for z in corpus]
    return np.concatenate(vecs)


def train(x_train):
    model_dm = Doc2Vec(x_train, min_count=1, window=3,
                       sample=1e-3, negative=5, workers=4)
    model_dm.train(x_train, total_examples=model_dm.corpus_count, epochs=70)
    model_dm.save('model_dm')

    return model_dm


def infer(test_text):
    model_dm = Doc2Vec.load("model_dm")
    infer_vector_dm = model_dm.infer_vector(test_text)
    sims = model_dm.dv.most_similar([infer_vector_dm], topn=10)

    return sims

In [9]:
x_train = get_datasest()
x_train[:1]

docs size: 4


[TaggedDocument(words=['1,本报记者', '发自', '上海', '国外', '媒体', '昨日', '报道', '澳大利亚', '银行', 'acq', 'arie', '预计', '推出', '中国', '人民币', '10', '亿元', '商业', '住房', '抵押', '贷款', '资产', '证券化', '计划', '有关部门', '批准', '将是', '海外', '资金', '首次', '此项', '计划', '市场分析', '人士', '计划', '预计', '中国', '监管部门', '阻力', '考虑到', '交易', '相关', '高昂', '固定成本', '人民币', '10', '亿元', '可能是', '最低', '金额', '银行', '原本', '计划', '2006', '年初', '中国', '推出', 'macquarie', 'anda', '房地产', '投资信托', '计划', '香港特区', '证监会', '否决', '该银行', '中国', '房地产投资', '基金', '首席', '投资', '执行官', '此前', '开发商', '行列', '竟是', '金融机构', '项目', '投融资', '资本运作', '才是', '特长'], tags=[0])]

In [10]:
model_dm = train(x_train)
model_dm

In [16]:
test_text = ['上海', '国外', '媒体', '昨日', '报道', '澳大利亚', '银行', 'acq',
             '开发商', '行列', '竟是', '金融机构', '项目', '投融资', '资本运作', ]

sims = infer(test_text)
sims

[(0, 0.9996713399887085),
 (2, 0.9970905184745789),
 (3, 0.9969831109046936),
 (1, 0.9913798570632935)]

In [17]:
for index, sim_score in sims:
    sentence = x_train[index]
    words = ''
    for word in sentence[0]:
        words = words + word + ' '
    print(sim_score, words)

0.9996713399887085 1,本报记者 发自 上海 国外 媒体 昨日 报道 澳大利亚 银行 acq arie 预计 推出 中国 人民币 10 亿元 商业 住房 抵押 贷款 资产 证券化 计划 有关部门 批准 将是 海外 资金 首次 此项 计划 市场分析 人士 计划 预计 中国 监管部门 阻力 考虑到 交易 相关 高昂 固定成本 人民币 10 亿元 可能是 最低 金额 银行 原本 计划 2006 年初 中国 推出 macquarie anda 房地产 投资信托 计划 香港特区 证监会 否决 该银行 中国 房地产投资 基金 首席 投资 执行官 此前 开发商 行列 竟是 金融机构 项目 投融资 资本运作 才是 特长 
0.9970905184745789 2,美国 太空 网站 4月 27日 报道 5月 12日 14日 之间 73p 瓦斯 3号 彗星 30 碎片 史无前例 地球 对此 美国 宇航局 科学家 反驳 碎片 撞击 地球 更不 会引起 大规模 海啸 生物 灭绝 灾难 美国 宇航局 科学家 5月 12日 5月 28日 之间 即便是 73p 瓦斯 3号 彗星 最接近 地球 轨道 距离 地球 碎片 地球 月球 距离 20 多倍 不会有 危险 科学家 提醒 利用 会对 彗星 观察 科学家 预计 碎片 中最 明亮 碎片 双筒望远镜 肉眼 观察到 n101 
0.9969831109046936 3,化妆品 改善 皮肤 状况 表皮 角质化 过程 所需 时间 化妆品 三个月 会把 理想 皮肤 安全地 改善 预期 短期 化妆品 都是 加了 违禁 原料 皮肤 虽然在 天内 改善 很可能 导致 皮肤病 2. 植物 绿色 化妆品 作成 形态 装在 瓶子 出售 化妆品 不可能 不含 防腐剂 化学成分 迷信 化妆品 植物 纯天然 宣传 3. 化妆品 质量 越好 对照 成分 也许 发现 的产品 便宜 的产品 相差无几 的产品 配方 便宜 选购 化妆品 简单 办法 尝试 检测 合格 品牌 选择 不良反应 感觉 最舒服 那一 
0.9913798570632935 2,复旦 新浪 本报记者 杨国强 1984年 相貌端正 复旦大学 新闻系 大学 同学 回忆说 内向 做事 很有 生活 学习 很有 计划性 大学毕业 上海 电视台 当了 两年 记者 赴美 求学 先在 奥克拉荷 大学 拿了 新闻

本次使用的数据集为样本较少，样本内容比较单一，所以训练出来的结果区分度不大，我线下用2000条情感分析样本效果还可以。doc2vec提取句子向量用于情感分析是可行的。

In [18]:
import os
os.remove('model_dm')

本节完。